---
title: Create STAC metadata for MUR SST 
description: Tutorial for creating STAC metadata for a collection in CMR
author: Aimee Barciauskas
date: January 18, 2024
execute:
  freeze: true
  cache: true
---

## Run this notebook

You can launch this notebook in VEDA JupyterHub by clicking the link below.

[Launch in VEDA JupyterHub (requires access)](https://nasa-veda.2i2c.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/notebooks/veda-operations/publish-cmip6-kerchunk-stac.ipynb&branch=main) 

<details><summary>Learn more</summary>
    
### Inside the Hub

This notebook was written on a VEDA JupyterHub instance

See (VEDA Analytics JupyterHub Access)[https://nasa-impact.github.io/veda-docs/veda-jh-access.html] for information about how to gain access.

### Outside the Hub

You are welcome to run this anywhere you like (Note: alternatively you can run this on https://daskhub.veda.smce.nasa.gov/, MAAP, locally, ...), just make sure that the data is accessible, or get in contact with the VEDA team to enable access.

</details>

## Approach

This notebook creates STAC collection metadata for the [MUR SST](https://search.earthdata.nasa.gov/search/granules?p=C1996881146-POCLOUD) dataset. 

## Step 1: Install and import necessary libraries

In [1]:
%%capture
!pip install xstac

In [2]:
import earthaccess
import json
from datetime import datetime
import pandas as pd
import pystac
import requests
import s3fs
import xstac
import xarray as xr

## Step 2: Get Collection metadata from CMR

In [3]:
earthaccess.login()

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
No .netrc found in /home/jovyan


Enter your Earthdata Login username:  aimeeb
Enter your Earthdata password:  ········


You're now authenticated with NASA Earthdata Login
Using token with expiration date: 03/12/2024
Using user provided credentials for EDL


In [4]:
short_name = 'MUR-JPL-L4-GLOB-v4.1'
collection_query = earthaccess.collection_query()
r = collection_query.short_name(short_name)
cmr_collection = r.get(1)[0]

Pick out one granule to open for data cube dimensions and variables

In [5]:
first_result = earthaccess.search_data(
    short_name='MUR-JPL-L4-GLOB-v4.1',
    cloud_hosted=True,
    count=1
)

Granules found: 7902


### Sidebar: S3 vs HTTPS access

Right now, earthaccess only supports `open` over https. Below, we see that open the dataset with direct S3 access is many times faster.

In [6]:
files = earthaccess.open(first_result)

 Opening 1 granules, approx size: 0.0 GB


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
%%time
ds_https = xr.open_mfdataset(files)

CPU times: user 2.46 s, sys: 121 ms, total: 2.58 s
Wall time: 5.31 s


In [8]:
s3_link = first_result[0].data_links(access='direct')[0]

In [9]:
%%time
fs = s3fs.S3FileSystem(anon=False)
# Open the dataset using xarray
# The 'chunks' parameter enables Dask for lazy loading
ds_s3 = xr.open_dataset(fs.open(s3_link), engine='h5netcdf', chunks={})

CPU times: user 208 ms, sys: 32.5 ms, total: 241 ms
Wall time: 451 ms


## Step 3: Generate STAC metadata

The spatial and temporal extents are extracted from the CMR collection metadata.

In [10]:
spatial_extent = cmr_collection['umm']['SpatialExtent']
bounding_rectangle = spatial_extent['HorizontalSpatialDomain']['Geometry']['BoundingRectangles'][0]
extent_list = [
    bounding_rectangle["WestBoundingCoordinate"],
    bounding_rectangle["SouthBoundingCoordinate"],   
    bounding_rectangle["EastBoundingCoordinate"],
    bounding_rectangle["NorthBoundingCoordinate"],    
]
spatial_extent = list(map(int, extent_list))

temporal_extent = cmr_collection['umm']['TemporalExtents'][0]['RangeDateTimes'][0]
start = temporal_extent['BeginningDateTime']
end = temporal_extent.get('EndingDateTime', None)

extent = pystac.Extent(
    spatial=pystac.SpatialExtent(bboxes=[spatial_extent]),
    temporal=pystac.TemporalExtent([[pd.to_datetime(start), pd.to_datetime(end)]])
)

Add the provider information from CMR.

In [11]:
cmr_roles_to_pystac_roles = {
    'PROCESSOR': pystac.ProviderRole.PROCESSOR,
    'DISTRIBUTOR': pystac.ProviderRole.HOST
}
def create_providers_from_data_centers(data_centers):
    providers = []

    for center in data_centers:
        # Extracting necessary information from each data center
        short_name = center.get("ShortName", "")
        long_name = center.get("LongName", "")
        roles = []
        for role in center.get("Roles", []):
            if role in cmr_roles_to_pystac_roles:
                roles.append(cmr_roles_to_pystac_roles[role])
        url = next((url_info["URL"] for url_info in center.get("ContactInformation", {}).get("RelatedUrls", []) 
                    if url_info.get("URLContentType") == "DataCenterURL"), None)

        # Creating a PySTAC Provider object
        provider = pystac.Provider(name=short_name, description=long_name, roles=roles, url=url)
        providers.append(provider)

    return providers

data_centers = cmr_collection['umm']['DataCenters']
providers = create_providers_from_data_centers(data_centers)

Put it all together to intialize a `pystac.Collection` instance.

In [22]:
_id = short_name.replace('.', '_')
description = cmr_collection['umm']['Abstract']
concept_id = cmr_collection['meta']['concept-id']
collection = pystac.Collection(
    id=_id,
    extent=extent,
    description=cmr_collection['umm']['Abstract'],
    providers=providers,
    stac_extensions=['https://stac-extensions.github.io/datacube/v2.0.0/schema.json'],
    license="CC0-1.0",
    extra_fields={'collection_concept_id': concept_id}
)

That collection instance is used by `xstac` to generate additional metadata, specifically for the [`datacube extension`](https://github.com/stac-extensions/datacube) information.

In [23]:
collection_template = collection.to_dict()

# see https://github.com/stac-utils/xstac/issues/30
for k, v in ds_s3.variables.items():
    attrs = {name: xr.backends.zarr.encode_zarr_attr_value(value) for name, value in v.attrs.items()}
    ds_s3[k].attrs = attrs
        
collection = xstac.xarray_to_stac(
    ds_s3,
    collection_template,
    temporal_dimension='time',
    x_dimension="lon",
    y_dimension="lat",
    # TODO: get this from attributes if possible
    reference_system="4326",
    validate=False
)
# It should validate, but it doesn't :(
# TODO: investigate why this is not validating.
# TypeError: Object of type int16 is not JSON serializable
collection.validate()

['https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json',
 'https://stac-extensions.github.io/datacube/v2.0.0/schema.json']

Set the second value for the time extent to `None` since the dataset is ongoing. Otherwise the extent is just the extent of the first file in the collection.

In [24]:
collection.to_dict()['cube:dimensions']['time']['extent'][1] = None

Add [renders](https://github.com/stac-extensions/render) extension.

In [26]:
collection.extra_fields['renders'] = {
    "analysed_sst": {
      "title": "Renders configuration for sea surface temperature",
      "resampling": "average",
      "colormap_name": "ocean",
      "rescale": [[271,305]]
    }
}
collection.to_dict()['cube:variables']['analysed_sst']['renders'] = "analysed_sst"

## Step 4: Write to json

In [27]:
with open(f"{collection.id}.stac.json", "w+") as f:
    f.write(json.dumps(collection.to_dict()))